---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

def loadDatasets():
    trainDS = pd.read_csv('train.csv',engine='python')
    testDs = pd.read_csv('test.csv',engine='python')
    return (trainDS, testDs)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report,roc_curve, auc

def blight_model():
    
    trainDS, testDs = loadDatasets()
    cols = [
    "fine_amount",
    "admin_fee",
    "state_fee",
    "late_fee",
    "discount_amount",
    "clean_up_cost",
    "judgment_amount",
    ]


    trainDsNoNa =trainDS[~np.isnan(trainDS["compliance"])]

    X = trainDsNoNa[cols]
    Y = trainDsNoNa["compliance"]

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y)

    lr = LogisticRegression(random_state=0).fit(Xtrain, Ytrain)
    y_score_lr = lr.decision_function(Xtest)
    lr_predicted = lr.predict(Xtest)
    result = lr.predict_proba(testDs[cols])

    #print('Accuracy: {:.2f}'.format(accuracy_score(Ytest, lr_predicted)))
    #print('Precision: {:.2f}'.format(precision_score(Ytest, lr_predicted)))
    #print('Recall: {:.2f}'.format(recall_score(Ytest, lr_predicted)))
    #print('F1: {:.2f}'.format(f1_score(Ytest, lr_predicted)))
    #print(classification_report(Ytest, lr_predicted, target_names = ['not 1', '1']))

    fpr_lr, tpr_lr, _ = roc_curve(Ytest, y_score_lr)
    auc(fpr_lr, tpr_lr)
    return pd.Series(result[:,1], index=testDs["ticket_id"])
    #return result



In [8]:
result = blight_model()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
import numpy as np
bm = blight_model()
res = '{:40s}'.format('Object Type:')
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += '{:40s}'.format('Data Shape:')
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += '{:40s}'.format('Data Values Type:')
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += '{:40s}'.format('Data Values Infinity:')
res += ['Failed: values should not be infinity\n','Passed\n'][not any(np.isinf(bm))]
res += '{:40s}'.format('Data Values NaN:')
res += ['Failed: values should not be NaN\n','Passed\n'][not any(np.isnan(bm))]
res += '{:40s}'.format('Data Values in [0,1] Range:')
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += '{:40s}'.format('Data Values not all 0 or 1:')
res += ['Failed: values should be scores not predicted labels\n','Passed\n'][not all((bm.isin({0,1,0.0,1.0})))]
res += '{:40s}'.format('Index Type:')
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += '{:40s}'.format('Index Values:')
if bm.index.shape==(61001,):
    res +=['Failed: index values should match test.csv\n','Passed\n'
          ][all(pd.read_csv('test.csv',usecols=[0],index_col=0
                           ).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed: bm.index length should be 61001'
res += '{:40s}'.format('Can run model twice:')
bm2 = None
try:
    bm2 = blight_model()
    res += 'Passed\n'
except:
    res += ['Failed: second run of blight_model() threw an Exception']
res += '{:40s}'.format('Can run model twice with same results:')
if not bm2 is None:
    res += ['Failed: second run of blight_model() produced different results (this might not be a problem)\n','Passed\n'][
        all(bm.apply(lambda x:round(x,3))==bm2.apply(lambda x:round(x,3))) and all(bm.index==bm2.index)]    
print(res)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Object Type:                            Passed
Data Shape:                             Passed
Data Values Type:                       Passed
Data Values Infinity:                   Passed
Data Values NaN:                        Passed
Data Values in [0,1] Range:             Passed
Data Values not all 0 or 1:             Passed
Index Type:                             Passed
Index Values:                           Passed
Can run model twice:                    Passed
Can run model twice with same results:  Failed: second run of blight_model() produced different results (this might not be a problem)



In [6]:
def testing():
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import train_test_split
    import matplotlib.pyplot as plt
    X = trainDsNoNa[trainingCols]
    Y = trainDsNoNa["compliance"]

    Xtrain,Xtest, Ytrain, Ytest = train_test_split(X,Y)

    #pd.get_dummies( Xtrain)
    m = DecisionTreeClassifier(max_depth=3).fit(Xtrain,Ytrain)
    m.score(Xtest,Ytest)
    print(Xtrain.columns)
    m.feature_importances_ 
    #Ytest.tail()        
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score    
    tree_predicted = m.predict(Xtest)
    #print('Accuracy: {:.2f}'.format(accuracy_score(Ytest, tree_predicted)))
    #print('Precision: {:.2f}'.format(precision_score(Ytest, tree_predicted)))
    #print('Recall: {:.2f}'.format(recall_score(Ytest, tree_predicted)))
    #print('F1: {:.2f}'.format(f1_score(Ytest, tree_predicted)))
    #
    
    trainingCols = [
    #"ticket_id",
    #"agency_name",
    #"inspector_name",
    #"violator_name",
    #"ticket_issued_date",
    #"hearing_date",
    #"violation_code",
    #"disposition",


    #"violation_street_number",
    #"mailing_address_str_number",
    "fine_amount",
    "admin_fee",
    "state_fee",
    "late_fee",
    "discount_amount",
    "clean_up_cost",
    "judgment_amount",
    #"grafitti_status"
    ]
    testingCols = ["payment_amount",
    "payment_date",
    "payment_status",
    "balance_due",
    "collection_status",
    "compliance",
    "compliance_detail",
    #]
    "ticket_issued_date",
    "fine_amount",
    "admin_fee",
    "state_fee",
    "late_fee",
    "discount_amount",
    "clean_up_cost",
    "judgment_amount",
    "grafitti_status",
    "hearing_date"]